# A Sample of Data Augmentation
---
Zhiang Chen, Nov 2018 @ DREAMS

### 1. Generate Masks from XML
- Download the xml annotations and original images from LabelMe
- Create dataset folders under directory "datasets"  
e.g. a tornado datasets
```
--data_augmentator
|--datasets  
    |--rocks
        |--annotation: rocks1.xml, rocks2.xml, ...
        |--image: rocks1.jpg, rocks2.jpg, ...
```
- Genrate masks from XML files

In [4]:
from xml2mask import polygonReader
objects = ['drone'] # give a list of the annotated objects
pr = polygonReader("drone", objects)

#save compact masks on .jpg files
#pr.saveMask(dim=(4000,4000))

#generate and save multi-channel images on .npy files
pr.generateMask2(dim=(1024, 768), resize=(768, 1024), saveOnline=True) 

177it [00:01, 158.58it/s]


### 2. Augment data
Before running the script, create a folder and move .npy files into the folder. And create a folder for augmentation data.
```
--data_augmentator
|--datasets
    |--rocks
        |--annotation: rocks1.xml, rocks2.xml, ...
        |--image: rocks1.jpg, rocks2.jpg, ...
        |--npy: rocks1.npy, rocks2.npy, ...
        |--aug: augmentation data
```
Resize the data before augmentation.

In [4]:
from augmentation import augmentor
import numpy as np
from tqdm import tqdm
import cv2
import uuid

In [7]:
config = dict(
            mode=2, 
            resize_dim=(500, 500),
            batch_number=10,
            rotation_min=-90,
            rotation_max=90,
            fliplr=True,
            flipud=True,
            zoom_min=0.8,
            zoom_max=1.2)

image_path = './datasets/Crater/image/'
annotation_path = './datasets/Crater/npy/'
aug_path = './datasets/Crater/aug/'
aug = augmentor(image_path, annotation_path, **config)

for i,m,f in tqdm(aug):
    unid = uuid.uuid4().hex
    img_name = aug_path + f + "_" + unid + '.png'
    ann_name = aug_path + f + "_" + unid + '.npy'
    cv2.imwrite(img_name, i)
    np.save(ann_name, m)

0it [00:00, ?it/s]


error: /build/opencv-L2vuMj/opencv-3.2.0+dfsg/modules/imgproc/src/imgwarp.cpp:5028: error: (-215) ifunc != 0 in function remap


In [ ]:
config = dict(
            mode=2, 
            resize_dim=(1000, 1000),
            batch_number=10,
            rotation_min=-90,
            rotation_max=90,
            fliplr=True,
            flipud=True,
            zoom_min=0.8,
            zoom_max=1.2)

image_path = './datasets/drone/image/'
annotation_path = './datasets/drone/npy/'
aug_path = './datasets/drone/aug/'
aug = augmentor(image_path, annotation_path, **config)

for i,m,f in tqdm(aug):
    unid = uuid.uuid4().hex
    img_name = aug_path + f + "_" + unid + '.jpg'
    ann_name = aug_path + f + "_" + unid + '.npy'
    cv2.imwrite(img_name, i)
    np.save(ann_name, m)

### 3. Rename images and annotations by numerical order

It is recommended to back up the data before renaming.

In [2]:
from rename import renamer
rn = renamer('./datasets/rocks/image/', './datasets/rocks/annotation/')

In [3]:
rn.rename(mode=1, image2png=True, annotation_suffix='.npy', annotation2suffix='.npy')

Alright, data augmentation done.